---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [13]:
def graph_identification():
    # We will compare clustering coefficient, average path length, 
    #degree distribution for all of them and check for power laws
    
    import matplotlib.pyplot as plt
    
    #Graph 1
    G1 = P1_Graphs[0]
    cc1 = nx.average_clustering(G1) #0.03
    
    apl1 = nx.average_shortest_path_length(G1) #4.09
    
    degrees1 = G1.degree()
    degree_values1 = sorted(set(degrees1.values()))
    histogram1 = [list(degrees1.values()).count(i)/float(nx.number_of_nodes(G1)) for i in degree_values1] 
    #high prob of low degree nodes, low prob of high degree nodes
    #plt.plot(degree_values1,histogram1,'o')
    #plt.xscale('log')
    #plt.yscale('log')
    #plt.show() #Power-Law
    
    #Graph 1 is PA
    
    #Graph 2
    G2 = P1_Graphs[1]
    cc2 = nx.average_clustering(G2) #0.56
    
    apl2 = nx.average_shortest_path_length(G2) #5.09
    
    degrees2 = G2.degree()
    degree_values2 = sorted(set(degrees2.values()))
    histogram2 = [list(degrees2.values()).count(i)/float(nx.number_of_nodes(G2)) for i in degree_values2] 
    #high prob of low degree nodes, low prob of high degree nodes
    #plt.plot(degree_values2,histogram2,'o')
    #plt.xscale('log')
    #plt.yscale('log')
    #plt.show() #No Power-Law
    
    #Graph 2 is SW_L
    
    #Graph 3
    G3 = P1_Graphs[2]
    cc3 = nx.average_clustering(G3) #0.4
    
    apl3 = nx.average_shortest_path_length(G3) #9.38
    
    degrees3 = G3.degree()
    degree_values3 = sorted(set(degrees3.values()))
    histogram3 = [list(degrees3.values()).count(i)/float(nx.number_of_nodes(G3)) for i in degree_values3] 
    #high prob of low degree nodes, low prob of high degree nodes
    #plt.plot(degree_values3,histogram3,'o')
    #plt.xscale('log')
    #plt.yscale('log')
    #plt.show() #No Power-Law
    
    #Graph 3 is SW_L
    
    #Graph 4
    G4 = P1_Graphs[3]
    cc4 = nx.average_clustering(G4) #0.04
    
    apl4 = nx.average_shortest_path_length(G4) #3.10
    
    degrees4 = G4.degree()
    degree_values4 = sorted(set(degrees4.values()))
    histogram4 = [list(degrees4.values()).count(i)/float(nx.number_of_nodes(G4)) for i in degree_values4] 
    #high prob of low degree nodes, low prob of high degree nodes
    #plt.plot(degree_values4,histogram4,'o')
    #plt.xscale('log')
    #plt.yscale('log')
    #plt.show() #Power-Law
    
    #Graph 4 is PA
    
    #Graph 5
    G5 = P1_Graphs[4]
    cc5 = nx.average_clustering(G5) #0.003
    
    apl5 = nx.average_shortest_path_length(G5) #5.08
    
    degrees5 = G5.degree()
    degree_values5 = sorted(set(degrees5.values()))
    histogram5 = [list(degrees5.values()).count(i)/float(nx.number_of_nodes(G5)) for i in degree_values5] 
    #high prob of low degree nodes, low prob of high degree nodes
    #plt.plot(degree_values5,histogram5,'o')
    #plt.xscale('log')
    #plt.yscale('log')
    #plt.show() #No Power-Law
    
    #Graph 5 is SW_H
    
    return ['PA','SW_L','SW_L','PA','SW_H']

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [14]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [81]:
def salary_predictions():
    from sklearn.metrics import roc_curve, auc
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    from sklearn import preprocessing
    
    #Create dataframe of features
    df = pd.DataFrame(index=G.nodes())
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['Closeness'] = pd.Series(nx.closeness_centrality(G))
    df['Betweenness'] = pd.Series(nx.betweenness_centrality(G,normalized=True,endpoints=False))
    df['Hub'] = pd.Series(nx.hits(G)[0])
    df['Authority'] = pd.Series(nx.hits(G)[1])
    df['Degree'] = pd.Series(nx.degree(G))
    df['PageRank'] =pd.Series(nx.pagerank(G,alpha=0.85))
    
    #Split by NaN values in ManagementSalary to isolate the unknown values
    notnull = df[pd.notnull(df['ManagementSalary'])]
    X_notnull = notnull.iloc[:,1:7]
    y_notnull = notnull['ManagementSalary']
    null = df[pd.isnull(df['ManagementSalary'])]
    X_pred = null.iloc[:,1:7]
    
    ###MODEL SELECTION CODE
    #Split notnull into train and test to check auc scores 
    #X_train, X_test, y_train, y_test = train_test_split(X_notnull, y_notnull, test_size=0.2)
    #scaler = preprocessing.StandardScaler().fit(X_train)
    #X_train = scaler.transform(X_train)  
    #X_test = scaler.transform(X_test)  
    #X_pred = scaler.transform(X_pred)
    
    #Use Logistic regression on split data 
    #logRegClassifier = LogisticRegression()
    #logRegClassifier.fit(X_train, y_train)
    
    #Check AUC score 
    #fpr, tpr, _ = roc_curve(y_test, logRegClassifier.decision_function(X_test))
    #roc_auc = auc(fpr, tpr)
    ###
    
    ###FINAL MODEL CODE
    scaler = preprocessing.StandardScaler().fit(X_notnull)
    X_train = scaler.transform(X_notnull)   
    X_pred = scaler.transform(X_pred)
    
    logRegClassifier = LogisticRegression()
    model = logRegClassifier.fit(X_train, y_notnull)
    
    return pd.Series(model.predict_proba(X_pred)[:,1],index=null.index)

salary_predictions()

1       0.130315
2       0.601843
5       0.999929
8       0.091910
14      0.231920
18      0.378318
27      0.229576
30      0.245308
31      0.118949
34      0.050890
37      0.052258
40      0.079390
45      0.177313
54      0.099462
55      0.232166
60      0.051095
62      0.999024
65      0.914992
77      0.121383
79      0.185384
97      0.052247
101     0.075204
103     0.117750
108     0.152344
113     0.403503
122     0.041398
141     0.399653
142     0.766276
144     0.080203
145     0.522989
          ...   
913     0.041712
914     0.052667
915     0.011648
918     0.054624
923     0.013677
926     0.041071
931     0.043528
934     0.032608
939     0.022274
944     0.014427
945     0.021210
947     0.027021
950     0.209693
951     0.020827
953     0.044447
959     0.021466
962     0.012588
963     0.149550
968     0.018571
969     0.021469
974     0.043196
984     0.030022
987     0.042041
989     0.019977
991     0.023028
992     0.027943
994     0.023653
996     0.0153

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [82]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [105]:
def new_connections_predictions():
    from sklearn.metrics import roc_curve, auc
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    from sklearn import preprocessing
    
    #Create dataframe of features
    df = future_connections
    df['Preferential Attachment'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
    df['Common Neighbors'] = df.index.map(lambda e: len(list(nx.common_neighbors(G, e[0], e[1]))))
    df['Jaccard Coefficient'] = [i[2] for i in nx.jaccard_coefficient(G, df.index)]
    df['Resource Allocation'] = [i[2] for i in nx.resource_allocation_index(G, df.index)]
    
    #Split by NaN values in Future Connection to isolate the unknown values
    notnull = df[pd.notnull(df['Future Connection'])]
    X_notnull = notnull.iloc[:,1:4]
    y_notnull = notnull['Future Connection']
    null = df[pd.isnull(df['Future Connection'])]
    X_pred = null.iloc[:,1:4]
    
    ###MODEL SELECTION CODE
    #Split notnull into train and test to check auc scores 
    #X_train, X_test, y_train, y_test = train_test_split(X_notnull, y_notnull, test_size=0.2)
    #scaler = preprocessing.StandardScaler().fit(X_train)
    #X_train = scaler.transform(X_train)  
    #X_test = scaler.transform(X_test)  
    #X_pred = scaler.transform(X_pred)
    
    #Use Logistic regression on split data 
    #logRegClassifier = LogisticRegression()
    #logRegClassifier.fit(X_train, y_train)
    
    #Check AUC score 
    #fpr, tpr, _ = roc_curve(y_test, logRegClassifier.decision_function(X_test))
    #roc_auc = auc(fpr, tpr)
    ###
    
    ###FINAL MODEL CODE
    scaler = preprocessing.StandardScaler().fit(X_notnull)
    X_train = scaler.transform(X_notnull)   
    X_pred = scaler.transform(X_pred)
    
    logRegClassifier = LogisticRegression()
    model = logRegClassifier.fit(X_train, y_notnull)
    
    return pd.Series(model.predict_proba(X_pred)[:,1],index=null.index)

new_connections_predictions()

(107, 348)    0.037380
(542, 751)    0.013991
(20, 426)     0.645978
(50, 989)     0.014264
(942, 986)    0.014561
(324, 857)    0.014226
(13, 710)     0.162431
(19, 271)     0.106555
(319, 878)    0.014359
(659, 707)    0.014019
(49, 843)     0.014445
(208, 893)    0.014085
(377, 469)    0.006034
(405, 999)    0.026002
(129, 740)    0.021820
(292, 618)    0.022448
(239, 689)    0.014378
(359, 373)    0.008657
(53, 523)     0.038916
(276, 984)    0.014431
(202, 997)    0.014489
(604, 619)    0.048722
(270, 911)    0.014373
(261, 481)    0.078423
(200, 450)    0.862039
(213, 634)    0.013908
(644, 735)    0.047868
(346, 553)    0.013247
(521, 738)    0.012043
(422, 953)    0.021364
                ...   
(672, 848)    0.014373
(28, 127)     0.974246
(202, 661)    0.013138
(54, 195)     0.999937
(295, 864)    0.014183
(814, 936)    0.013949
(839, 874)    0.014561
(139, 843)    0.014207
(461, 544)    0.011250
(68, 487)     0.011116
(622, 932)    0.014117
(504, 936)    0.018298
(479, 528) 